In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.environ.get("GROQ_API_KEY")



In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001715F4CB8E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001715F4C93F0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
response = model.invoke([HumanMessage(content = "My name is Snowfall")])
response


AIMessage(content="It's lovely to meet you, Snowfall! \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 14, 'total_tokens': 41, 'completion_time': 0.049090909, 'prompt_time': 0.001248299, 'queue_time': 0.24372749, 'total_time': 0.050339208}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3ed768a2-24c0-47f5-bf2d-a62fa7951e99-0', usage_metadata={'input_tokens': 14, 'output_tokens': 27, 'total_tokens': 41})

In [5]:

from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content = "My name is Snowfall"),
        AIMessage(content = "Hello Snowfall, it's nice to meet you! \n\nIs there anything I can help you with today?"),
        HumanMessage(content = "Whats my name?")
    ]
) #this is actually acting as the conversational history
# whether its able to remember the past history

AIMessage(content='Your name is Snowfall. 😊 \n\nI remember! How can I help you further?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 52, 'total_tokens': 74, 'completion_time': 0.04, 'prompt_time': 0.001713798, 'queue_time': 0.248349431, 'total_time': 0.041713798}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--563235cb-4a3c-4a73-863f-19b4cd86343b-0', usage_metadata={'input_tokens': 52, 'output_tokens': 22, 'total_tokens': 74})

message history
we can use a message history class to wrap our model and make it stateful, this will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this


In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id : str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # object of this chatmessagehistory class
    return store[session_id]  




In [8]:
config ={"configurable" :{
    "session_id":" chat1"
}} 

In [9]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [10]:
response = with_message_history.invoke(
    [
        HumanMessage(content = "My name is Snowfall")],
        config = config
)

In [11]:
response = with_message_history.invoke(
    [
        HumanMessage(content = "What's my name?")],
        config = config
)
response

AIMessage(content='Your name is Snowfall. 😊  \n\nI remember!  \n\nDo you have any other questions for me?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 53, 'total_tokens': 79, 'completion_time': 0.047272727, 'prompt_time': 0.001784851, 'queue_time': 0.250963839, 'total_time': 0.049057578}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--619d0143-3676-40bb-9629-3a09e0f24702-0', usage_metadata={'input_tokens': 53, 'output_tokens': 26, 'total_tokens': 79})

In [12]:
## Managing the conversation history
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy ="last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

